# Start default Node


In [24]:
# ./elasticsearch

In [ ]:
{
  "name" : "nXNCqGl",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "hIMiDMFCRp28NQd8UdiAYw",
  "version" : {
    "number" : "5.3.1",
    "build_hash" : "5f9cf58",
    "build_date" : "2017-04-17T15:52:53.846Z",
    "build_snapshot" : false,
    "lucene_version" : "6.4.2"
  },
  "tagline" : "You Know, for Search"
}

# Start Node with parameters

In [25]:
# ./elasticsearch -Ecluster.name=my_cluster_name -Enode.name=my_node_name

In [ ]:
{
  "name" : "my_node_name",
  "cluster_name" : "my_cluster_name",
  "cluster_uuid" : "hIMiDMFCRp28NQd8UdiAYw",
  "version" : {
    "number" : "5.3.1",
    "build_hash" : "5f9cf58",
    "build_date" : "2017-04-17T15:52:53.846Z",
    "build_snapshot" : false,
    "lucene_version" : "6.4.2"
  },
  "tagline" : "You Know, for Search"
}

## You can have multiple nodes in the same cluster.

Change the port (e.g. 9201) and run the script. If you have already defined the name of the cluster you have to use the same name otherwise it will fail to create the node.

## Get Cluster Health

In [30]:
# http://localhost:9200/_cluster/health?pretty

# curl -XGET 'localhost:9200/_cat/health?v&pretty' 

## Get Nodes

In [28]:
# curl -XGET 'localhost:9200/_cat/nodes?v&pretty'

## Get Indices

In [31]:
# curl -XGET 'localhost:9200/_cat/indices?v&pretty'

## Create/Delete Index

In [35]:
# curl -XPUT 'localhost:9200/index_name?pretty&pretty'

# curl -XDELETE 'localhost:9200/index_name?pretty&pretty'

## Add/Delete Data to Index

In [43]:
#########    With specific ID     ########

# curl -XPUT 'localhost:9200/customer/external/1?pretty&pretty' -H 'Content-Type: application/json' -d'
# {
#   "name": "Panos Sideris", "address": "London 101"
# }'

#######     Random generated ID    #######

# curl -XPOST 'localhost:9200/customer/external?pretty&pretty' -H 'Content-Type: application/json' -d'
# {
#   "name": "Donald Duck", "address": "Homeless"
# }'


##########################################

# curl -XDELETE 'localhost:9200/customer/external/1?pretty&pretty'

## Update Data to Index

In [46]:
# curl -XPOST 'localhost:9200/customer/external/1/_update?pretty&pretty' -H 'Content-Type: application/json' -d'
# {
#   "doc": { "wife name": "Mary" }
# }'


##########################################

# curl -XPOST 'localhost:9200/customer/external/_bulk?pretty&pretty' -H 'Content-Type: application/json' -d'
# {"index":{"_id":"1"}}
# {"name": "John Doe" }
# {"index":{"_id":"2"}}
# {"name": "Jane Doe" }'


#########################################

# curl -XPOST 'localhost:9200/customer/external/_bulk?pretty&pretty' -H 'Content-Type: application/json' -d'
# {"update":{"_id":"1"}}
# {"doc": { "name": "John Doe becomes Jane Doe" } }
# {"delete":{"_id":"2"}}'

## Import Data

In [ ]:
# curl -H "Content-Type: application/json" -XPOST 'localhost:9200/bank/account/_bulk?pretty&refresh' --data-binary "@accounts.json"

## Search Data

In [48]:
# curl -XGET 'localhost:9200/bank/_search?q=*&sort=account_number:asc&pretty&pretty'


########################################

# curl -XGET 'localhost:9200/bank/_search?pretty' -H 'Content-Type: application/json' -d'
# {
#   "query": { "match_all": {} },
#   "sort": [
#     { "account_number": "asc" }
#   ]
# }'

## Search with "Select From"

In [ ]:
# curl -XGET 'localhost:9200/bank/_search?pretty' -H 'Content-Type: application/json' -d'
# { 
#   "query": { "match_all": {} },
#   "_source": ["account_number", "balance"]
# }'

## Search with Boolean AND

In [1]:
# curl -XGET 'localhost:9200/bank/_search?pretty' -H 'Content-Type: application/json' -d'
# {
#   "query": {
#     "bool": {
#       "must": [
#         { "match": { "address": "mill" } },
#         { "match": { "address": "lane" } }
#       ]
#     }
#   }
# }'

## Search with Boolean OR

In [2]:
# curl -XGET 'localhost:9200/bank/_search?pretty' -H 'Content-Type: application/json' -d'
# {
#   "query": {
#     "bool": {
#       "should": [
#         { "match": { "address": "mill" } },
#         { "match": { "address": "lane" } }
#       ]
#     }
#   }
# }'

## Sort with script

In [2]:
POST /bp_en_gb,bp_en_us/specialty/_search
{
    "track_scores": true,
    "query" : {
        "term" : { "specialtyName" : "Vascular surgery" }
    },
    "sort" : [
        {"_script" : {
            "type" : "number",
            "script" : {
                "lang": "painless",
                "inline": "if(doc['_index'].value.contains('en_gb') ){return 0} else {return 1}"
            },
            "order" : "desc"
            }
        },
        "_score"
    
    ]
}

## search against multi-indexes, remove duplication by id and sort by indexName

In [2]:
POST /bp_en_gb,bp_en_us/monograph_specialty/_search
{
  "from":0,
  "size":0,
  "query":{
    "bool":{
      "should":[
        {
          "bool":{
            "must":[
              {
                "multi_match":{
                  "query":"Vascular surgery",
                  "fields":[
                    "specialtyName^1.0"
                  ],
                  "type":"cross_fields",
                  "operator":"AND",
                  "slop":0,
                  "prefix_length":0,
                  "max_expansions":50,
                  "lenient":false,
                  "zero_terms_query":"NONE",
                  "boost":1.0
                }
              }
            ],
            "disable_coord":false,
            "adjust_pure_negative":true,
            "boost":1.0
          }
        }
      ],
      "disable_coord":false,
      "adjust_pure_negative":true,
      "boost":1.0
    }
  },
  "_source":{
    "includes":[
      "id",
      "specialtyId",
      "specialtyName"
    ],
    "excludes":[

    ]
  },
  "aggs":{
    "dedup":{
      "terms":{
        "field":"id"
      },
      "aggs":{
        "dedup_docs":{
          "top_hits":{
            "size":2,
            "sort":[
              {
                "_script":{
                  "script":{
                    "inline":"if(doc['_index'].value.contains('en_gb') ){return 0} else {return 1}",
                    "lang":"painless"
                  },
                  "type":"number",
                  "order":"asc"
                }
              }
            ]
          }
        }
      }
    }
  }
}

## suggestion_analyzer for index setting

In [2]:
POST /bp_en_us/_settings
{
  "analysis":{
    "filter":{
      "default_synonym_filter":{
        "type":"synonym",
        "synonyms_path":"synonym/MESH_default_v1.0.txt",
        "ignore_case":"true"
      },
      "snowball_filter":{
        "type":"snowball",
        "language":"English"
      },
      "shingle_filter":{
        "min_shingle_size":2,
        "max_shingle_size":3,
        "type":"shingle"
      }
    },
    "analyzer":{
      "default":{
        "language":"English",
        "tokenizer":"standard",
        "filter":[
          "default_synonym_filter",
          "snowball_filter",
          "asciifolding",
          "lowercase"
        ]
      },
      "suggestion_analyzer":{
        "tokenizer":"lowercase",
        "filter":[
          "lowercase",
          "asciifolding",
          "shingle_filter"
        ],
        "type":"custom"
      }
    }
  }
}

## mapping for auto completion

In [2]:
{
  "item":{
    "properties":{
      "id":{
        "type":"keyword"
      },
      "title":{
        "type":"text",
        "fields":{
          "raw":{
            "type":"keyword"
          },
          "suggest":{
            "type":"completion"
          }
        }
      }
    }
  }
}